<h1><center>Main Notebook - Random Forest</center></h1>

# Imports & Setup

In [ ]:
%load_ext autoreload
%autoreload 2
from utils import *
import utils.transformers.transformers as tr
import utils.transformers.sk4pandas as s4p

import os
import pickle
import urllib.request

import git
from IPython.core.magic import register_cell_magic
import mlflow
import pandas as pd
from sklearn.base import clone
from sklearn.compose import make_column_selector
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from scipy.stats import uniform, loguniform

git_repo = git.Repo(os.getcwd())

@register_cell_magic
def run_and_save(line, cell):
    'Run and save python code block to a file'
    with open(line, 'wt') as fd:
        fd.write(cell)
    code = compile(cell, line, 'exec')
    exec(code, globals())

# Init Datasets

In [ ]:
raw_main_df = pd.read_csv('data/train.csv')
raw_sub_df = pd.read_csv('data/test.csv')
raw_train_df, raw_test_df = train_test_split(raw_main_df, test_size=100, random_state=42)

X_all, y_all = raw_main_df.drop('Survived', axis=1), raw_main_df['Survived']
X_train, y_train = raw_train_df.drop('Survived', axis=1), raw_train_df['Survived']
X_test, y_test = raw_test_df.drop('Survived', axis=1), raw_test_df['Survived']

raw_train_df.sample(5, random_state=42)

In [ ]:
raw_train_df.dtypes

# Prepare Pipeline

In [ ]:
%%run_and_save blueprints/pipeline.py


clip_outliers = [
    (
        'Float',
        tr.ClipOutliers(std_band=3),
        make_column_selector(dtype_include=['float64'])
    )
]

fillna = [
    (
        'Mean',
        s4p.SimpleImputer(strategy='mean'),
        make_column_selector(dtype_include=['float64'])
    ),
    (
        'Most Frequent',
        s4p.SimpleImputer(strategy='most_frequent'),
        make_column_selector(dtype_include=['int64', 'object'])
    )
]

standardize = [
    (
        'Floats Ints',
        s4p.StandardScaler(),
        ['Age', 'Fare']
    )
]


steps_main = [
    ('Format Cabins', tr.Cabin() ),
    ('Set working columns', tr.SetupFeatures(cols_ignore=['PassengerId', 'Name', 'Ticket']) ),
    ('Clip Outliers', s4p.ColumnTransformer(clip_outliers, remainder='passthrough') ),
    ('Standardize', s4p.ColumnTransformer(standardize, remainder='passthrough')),
    ('Fill NaN', s4p.ColumnTransformer(fillna, remainder='passthrough') ),
    ('Convert Types', tr.AsTypes(coltypes_overwrite={'Age': 'float64'}) ),
    ('One Hot', s4p.OneHotEncoder(cols_select=['Pclass', 'Sex', 'Cabin', 'SibSp', 'Embarked']) ),
    ('Random Forest', RandomForestClassifier())
]

pipe = Pipeline(steps_main)

# Grid Params

In [ ]:
%%run_and_save blueprints/grid_params.py


ml_params_distributions = {
    'Random Forest__n_estimators': [180, 190, 200, 210, 220, 230],
    'Random Forest__max_depth': [6, 8, 10, 15],
    'Random Forest__min_samples_split': [5, 10, 15, 20],
    'Random Forest__max_features': ['sqrt']
}

ml_params_distributions_WITH_btp = ml_params_distributions.copy()
ml_params_distributions_WITH_btp.update({
    'Random Forest__bootstrap': [True],
    'Random Forest__max_samples': [0.6, 0.8]
})
ml_params_distributions_WITHOUT_btp = ml_params_distributions.copy()
ml_params_distributions_WITHOUT_btp.update({
    'Random Forest__bootstrap': [False],
})

rs = RandomizedSearchCV (
    pipe,
    param_distributions = ml_params_distributions_WITH_btp,
    n_iter=100,
    n_jobs=-1,
    refit=True,
    cv=CVSplitter(5, 80),
    return_train_score=True,
)

# ML Flow Run

In [ ]:
mlflow.sklearn.autolog(max_tuning_runs=10)

with mlflow.start_run(run_name='RF - Use Parch, With bootstrap') as run:
    mlflow.set_tags(get_lastcommit_infos(git_repo))
    mlflow.log_artifact('blueprints/pipeline.py', 'blueprints')
    mlflow.log_artifact('blueprints/grid_params.py', 'blueprints')
    #pipe.fit(X_all, y_all)
    rs.fit(X_all, y_all)

# Refit best model

In [24]:
run_id = 'd982a63d94df42c9b923bd001529d02b'
url_local_model = f'file:///Users/i538262/Desktop/Developments/PERSO/KAGGLE%20Titanic/mlruns/0/{run_id}/artifacts/best_estimator/model.pkl'
with urllib.request.urlopen(url_local_model) as model_file:
    best_model = pickle.load(model_file)
    
best_model_allfit = clone(best_model)
best_model_allfit.fit(X_all, y_all)

accuracy_score(y_test, best_model_allfit.predict(X_test))

2022/04/24 17:13:55 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '9ef365ece10e46848757ac67eb0cf62a', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2022/04/24 17:13:55 WARNING mlflow.utils: Truncated the value of the key `steps`. Truncated value: `[('Format Cabins', Cabin()), ('Set working columns', SetupFeatures(cols_ignore=['PassengerId', 'Name', 'Ticket'])), ('Clip Outliers', ColumnTransformer(Float__cols_ignore=[], Float__cols_select=[],
                  Float__std_band=3,
            ...`
2022/04/24 17:13:55 WARNING mlflow.utils: Truncated the value of the key `Clip Outliers`. Truncated value: `ColumnTransformer(Float__cols_ignore=[], Float__cols_select=[],
                  Float__std_band=3,
                  transformers=[('Float', ClipOutliers(std_band=3),
                                 <sklearn.compose._column_transformer.make_col...`
2022/04/24 17:13:55 WARNING

0.84

# Submit models predictions

In [25]:
branch_name = get_lastcommit_infos(git_repo)['Branch']
submit(f'submissions/{branch_name}/{run_id}.csv', raw_sub_df.PassengerId, best_model_allfit.predict(raw_sub_df))

# Model insights

# Code tests